In [1]:
import os
import sys

import numpy as np
import open3d as o3d
import torch
from pyntcloud import PyntCloud

# from open3d.web_visualizer import draw

torch.set_num_threads(1)

from utils.sh_utils import eval_sh

# o3d.visualization.webrtc_server.enable_webrtc()

%load_ext autoreload
%autoreload 2
%matplotlib inline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
model_param, iter_num = torch.load("./output/mcmc_deneme/checkpoints/MCMCGaussian_NoCompressor30000.pth", map_location="cpu")
point_cloud_tensor = model_param[1]
degree = model_param[0]

spherical_harmonics = torch.cat((model_param[2], model_param[3]), dim=1)
shs_view = spherical_harmonics.transpose(1, 2).view(-1, 3, (degree + 1) ** 2)

color = eval_sh(degree, shs_view, torch.tensor([0, 0, 1]))

# Convert PyTorch tensor to NumPy array
point_cloud_np = point_cloud_tensor.detach().numpy()

# Create an Open3D point cloud from the NumPy array
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud_np)

# pcd = o3d.io.read_point_cloud("./output/Masked_40k.ply")  # Replace with the actual file path
N = len(pcd.points)

# fit to unit cube
pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()), center=pcd.get_center())
pcd.colors = o3d.utility.Vector3dVector(color.detach().numpy())

# # Visualize the point cloud (optional)
# o3d.visualization.draw_geometries([pcd])

# FROM Open3D
# cloud = PyntCloud.from_instance("open3d", pcd)

# cloud.plot(mesh=True, backend="threejs")


# octree = o3d.geometry.Octree(max_depth=11)
# octree.convert_from_point_cloud(pcd, size_expand=0.01)
# o3d.visualization.draw_geometries([octree])

# Create a visualization window and add the point cloud to it
visualizer = o3d.visualization.Visualizer()
visualizer.create_window(visible=False)
visualizer.add_geometry(pcd)

# draw(octree)

# # Zoom in by setting the camera parameters
# # You can modify the camera parameters to zoom in or out
# # For example, you can change the field of view (fov) of the camera
# # and the distance of the camera from the point cloud
# zoom_in = True  # Set to True to zoom in, False to zoom out
# if zoom_in:
#     # Decrease the field of view to zoom in
#     fov = (
#         visualizer.get_view_control()
#         .convert_to_pinhole_camera_parameters()
#         .intrinsic.intrinsic_matrix[0, 0]
#     )
#     fov *= 0.5  # You can adjust the factor to control the amount of zoom
# else:
#     # Increase the field of view to zoom out
#     fov = (
#         visualizer.get_view_control()
#         .convert_to_pinhole_camera_parameters()
#         .intrinsic.intrinsic_matrix[0, 0]
#     )
#     fov *= 2.0  # You can adjust the factor to control the amount of zoom

# # Set the new camera parameters
# # visualizer.get_view_control().convert_to_pinhole_camera_parameters().intrinsic.intrinsic_matrix[0, 0] = fov

# # Run the visualization
# visualizer.run()

# # Close the visualization window
# visualizer.destroy_window()

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW


False

In [5]:
import open3d as o3d
torus = o3d.geometry.TriangleMesh.create_torus()
sphere = o3d.geometry.TriangleMesh.create_sphere()
# create shortcut for draw
ev = o3d.visualization.ExternalVisualizer("tcp://127.0.0.1:51455", timeout=20000)
ev.draw([ {'geometry': sphere, 'name': 'sphere'},
           {'geometry': torus, 'name': 'torus'} ])

[Open3D INFO] Connection::send() recv failed with: Resource temporarily unavailable
[Open3D INFO] Connection::send() recv failed with: Resource temporarily unavailable


In [8]:
import os
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt


def custom_draw_geometry_with_camera_trajectory(pcd, camera_trajectory_path,
                                                render_option_path,
                                                output_path):
    custom_draw_geometry_with_camera_trajectory.index = -1
    custom_draw_geometry_with_camera_trajectory.trajectory =\
        o3d.io.read_pinhole_camera_trajectory(camera_trajectory_path)
    custom_draw_geometry_with_camera_trajectory.vis = o3d.visualization.Visualizer(
    )
    image_path = os.path.join(output_path, 'image')
    if not os.path.exists(image_path):
        os.makedirs(image_path)
    depth_path = os.path.join(output_path, 'depth')
    if not os.path.exists(depth_path):
        os.makedirs(depth_path)

    print("Saving color images in " + image_path)
    print("Saving depth images in " + depth_path)

    def move_forward(vis):
        # This function is called within the o3d.visualization.Visualizer::run() loop
        # The run loop calls the function, then re-render
        # So the sequence in this function is to:
        # 1. Capture frame
        # 2. index++, check ending criteria
        # 3. Set camera
        # 4. (Re-render)
        ctr = vis.get_view_control()
        glb = custom_draw_geometry_with_camera_trajectory
        if glb.index >= 0:
            print("Capture image {:05d}".format(glb.index))
            # Capture and save image using Open3D.
            vis.capture_depth_image(
                os.path.join(depth_path, "{:05d}.png".format(glb.index)), False)
            vis.capture_screen_image(
                os.path.join(image_path, "{:05d}.png".format(glb.index)), False)

            # Example to save image using matplotlib.
            '''
            depth = vis.capture_depth_float_buffer()
            image = vis.capture_screen_float_buffer()
            plt.imsave(os.path.join(depth_path, "{:05d}.png".format(glb.index)),
                       np.asarray(depth),
                       dpi=1)
            plt.imsave(os.path.join(image_path, "{:05d}.png".format(glb.index)),
                       np.asarray(image),
                       dpi=1)
            '''

        glb.index = glb.index + 1
        if glb.index < len(glb.trajectory.parameters):
            ctr.convert_from_pinhole_camera_parameters(
                glb.trajectory.parameters[glb.index])
        else:
            custom_draw_geometry_with_camera_trajectory.vis.destroy_window()

        # Return false as we don't need to call UpdateGeometry()
        return False

    vis = custom_draw_geometry_with_camera_trajectory.vis
    vis.create_window(visible=False)
    vis.add_geometry(pcd)
    vis.get_render_option().load_from_json(render_option_path)
    vis.register_animation_callback(move_forward)
    vis.run()



if not o3d._build_config['ENABLE_HEADLESS_RENDERING']:
    print("Headless rendering is not enabled. "
            "Please rebuild Open3D with ENABLE_HEADLESS_RENDERING=ON")
    exit(1)

sample_data = o3d.data.DemoCustomVisualization()
pcd = o3d.io.read_point_cloud(sample_data.point_cloud_path)
print("Customized visualization playing a camera trajectory. "
        "Press ctrl+z to terminate.")
custom_draw_geometry_with_camera_trajectory(
    pcd, sample_data.camera_trajectory_path, sample_data.render_option_path,
    'HeadlessRenderingOutput')

Headless rendering is not enabled. Please rebuild Open3D with ENABLE_HEADLESS_RENDERING=ON
Customized visualization playing a camera trajectory. Press ctrl+z to terminate.
Saving color images in HeadlessRenderingOutput/image
Saving depth images in HeadlessRenderingOutput/depth
[Open3D WARNING] Failed to initialize GLFW


AttributeError: 'NoneType' object has no attribute 'load_from_json'